In [ ]:
import sklearn
import sklearn.metrics
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas
import mplhep
import pickle
import awkward
import glob
import bz2
import os
import tqdm
import fastjet
import vector
import uproot

mplhep.style.use("CMS")

import sys
sys.path += ["../../mlpf/"]

import jet_utils
sys.path += ["../../mlpf/plotting/"]

from plot_utils import ELEM_LABELS_CMS, ELEM_NAMES_CMS
from plot_utils import CLASS_LABELS_CMS, CLASS_NAMES_CMS
from plot_utils import cms_label, sample_label
from plot_utils import pid_to_text

In [ ]:
# uproot.open("/local/joosep/mlpf/cms/20240823_simcluster/nopu/QCDForPF_14TeV_TuneCUETP8M1_cfi/root/pfntuple_1000000.root")["pfana/pftree"].keys()

In [ ]:
def load_tree(ttree):
    particles_pythia = ttree.arrays(["gen_pt", "gen_eta", "gen_phi", "gen_energy", "gen_pdgid", "gen_status", "gen_daughters"])
    particles_cp = ttree.arrays(["caloparticle_pt", "caloparticle_eta", "caloparticle_phi", "caloparticle_energy", "caloparticle_pid"])
    genjet = ttree.arrays(["genjet_pt", "genjet_eta", "genjet_phi", "genjet_energy"])
    genmet = ttree.arrays(["genmet_pt"])
    return awkward.Array({"pythia": particles_pythia, "cp": particles_cp, "genjet": genjet, "genmet": genmet})

In [ ]:
#Download from https://jpata.web.cern.ch/jpata/mlpf/cms/20240823_simcluster/nopu/

tts = [
    load_tree(uproot.open(fn)["pfana/pftree"]) for fn in glob.glob("/local/joosep/mlpf/cms/20240823_simcluster/nopu/TTbar_14TeV_TuneCUETP8M1_cfi/root/pfntuple_70000*.root")
]
tts = awkward.concatenate(tts, axis=0)

In [ ]:
particles_pythia = tts["pythia"]
particles_cp = tts["cp"]

#genjets are directly from CMSSW, have some cuts applied on the particles
#see e.g. https://github.com/cms-sw/cmssw/blob/master/RecoJets/Configuration/python/GenJetParticles_cff.py#L8
#ideally, we should apply the same cuts on the pythia particles in mask_pythia
genjets = vector.awk(awkward.Array(
    awkward.zip(
        {   
            "pt": tts["genjet"]["genjet_pt"],
            "eta": tts["genjet"]["genjet_eta"],
            "phi": tts["genjet"]["genjet_phi"],
            "energy": tts["genjet"]["genjet_energy"],
        }
    ), with_name="Mometum4D"
))
genmet = tts["genmet"]["genmet_pt"]

In [ ]:
b = np.logspace(-2,4,100)
plt.figure(figsize=(5,5))

abs_pid = np.abs(particles_pythia["gen_pdgid"])
mask_pythia = (particles_pythia["gen_status"]==1) & (np.abs(particles_pythia["gen_eta"])<5)
mask_pythia_nonu = (particles_pythia["gen_status"]==1) & (np.abs(particles_pythia["gen_eta"])<5) & (abs_pid!=12) & (abs_pid!=14) & (abs_pid!=16)
mask_cp = np.abs(particles_cp["caloparticle_eta"])<5

plt.hist(awkward.flatten(particles_pythia[mask_pythia_nonu]["gen_pt"]), bins=b, label="Pythia", histtype="step")
plt.hist(awkward.flatten(particles_cp[mask_cp]["caloparticle_pt"]), bins=b, label="CaloParticle", histtype="step")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Particle $p_T$ [GeV]")
plt.legend()

In [ ]:
b = np.linspace(-5,5,100)
plt.figure(figsize=(5,5))

plt.hist(awkward.flatten(particles_pythia[mask_pythia_nonu]["gen_eta"]), bins=b, label="Pythia", histtype="step")
plt.hist(awkward.flatten(particles_cp[mask_cp]["caloparticle_eta"]), bins=b, label="CaloParticle", histtype="step")

plt.xlabel("Particle $\eta$")
plt.legend()

In [ ]:
#Sum pT in event
b = np.linspace(0,1000,100)
plt.figure(figsize=(5,5))

plt.title("sum $p_T$ in event")
plt.hist2d(
    awkward.to_numpy(awkward.sum(particles_pythia[mask_pythia_nonu]["gen_pt"], axis=1)),
    awkward.to_numpy(awkward.sum(particles_cp[mask_cp]["caloparticle_pt"], axis=1)),
    bins=(b,b), cmap="hot_r", #norm=matplotlib.colors.LogNorm()
)
plt.plot([0,2000],[0,2000], color="black", lw=0.5)
plt.xlabel("Pythia, no nu")
plt.ylabel("CaloParticle")

## Jets

In [ ]:
jets_coll = {}
jets_coll["genjet"] = genjets
jet_constituents = {}

vec = vector.awk(
    awkward.zip(
        {   
            "pt": particles_pythia[mask_pythia_nonu]["gen_pt"],
            "eta": particles_pythia[mask_pythia_nonu]["gen_eta"],
            "phi": particles_pythia[mask_pythia_nonu]["gen_phi"],
            "energy": particles_pythia[mask_pythia_nonu]["gen_energy"],
        }
    )
)
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(vec.to_xyzt(), jetdef)
jets_coll["pythia_nonu"] = cluster.inclusive_jets(min_pt=3)
jet_constituents["pythia_nonu"] = cluster.constituent_index(min_pt=3)

vec = vector.awk(
    awkward.zip(
        {   
            "pt": particles_pythia[mask_pythia]["gen_pt"],
            "eta": particles_pythia[mask_pythia]["gen_eta"],
            "phi": particles_pythia[mask_pythia]["gen_phi"],
            "energy": particles_pythia[mask_pythia]["gen_energy"],
        }
    )
)
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(vec.to_xyzt(), jetdef)
jets_coll["pythia"] = cluster.inclusive_jets(min_pt=3)

vec = vector.awk(
    awkward.zip(
        {   
            "pt": particles_cp[mask_cp]["caloparticle_pt"],
            "eta": particles_cp[mask_cp]["caloparticle_eta"],
            "phi": particles_cp[mask_cp]["caloparticle_phi"],
            "energy": particles_cp[mask_cp]["caloparticle_energy"],
        }
    )
)
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(vec.to_xyzt(), jetdef)
jets_coll["cp"] = cluster.inclusive_jets(min_pt=3)
jet_constituents["cp"] = cluster.constituent_index(min_pt=3)

genjet_to_pythia = jet_utils.match_two_jet_collections(jets_coll, "genjet", "pythia", 0.1)
genjet_to_cp = jet_utils.match_two_jet_collections(jets_coll, "genjet", "cp", 0.1)
pythia_to_cp = jet_utils.match_two_jet_collections(jets_coll, "pythia", "cp", 0.1)
pythia_nonu_to_cp = jet_utils.match_two_jet_collections(jets_coll, "pythia_nonu", "cp", 0.1)

In [ ]:
plt.figure(figsize=(5,5))
b = np.logspace(0,4,100)
plt.hist(awkward.flatten(jets_coll["genjet"].pt), bins=b, histtype="step", lw=1, label="genjet")
plt.hist(awkward.flatten(jets_coll["pythia"].pt), bins=b, histtype="step", lw=1, label="Pythia")
plt.hist(awkward.flatten(jets_coll["cp"].pt), bins=b, histtype="step", lw=1, label="CaloParticle")
plt.xscale("log")
plt.xlabel("jet $p_T$ [GeV]")
plt.ylabel("number of jets")
plt.legend()

In [ ]:
plt.figure(figsize=(5,5))
b = np.logspace(-1,1,600)

plt.hist(
    awkward.flatten(
        (jets_coll["pythia"][genjet_to_pythia["pythia"]].pt / jets_coll["genjet"][genjet_to_pythia["genjet"]].pt)
    ), bins=b, histtype="step", lw=1, label="Pythia"
);

plt.hist(
    awkward.flatten(
        (jets_coll["cp"][genjet_to_cp["cp"]].pt / jets_coll["genjet"][genjet_to_cp["genjet"]].pt)
    ), bins=b, histtype="step", lw=1, label="CaloParticle"
);

plt.xscale("log")
plt.yscale("log")
plt.xlabel("jet $p_T$ / GenJet jet $p_T$")
plt.legend(loc=1, fontsize=12)
plt.axvline(1.0, color="black", ls="--", lw=0.5)

In [ ]:
plt.figure(figsize=(5,5))
b = np.logspace(-1,1,200)

plt.hist(
    awkward.flatten(
        (jets_coll["cp"][pythia_nonu_to_cp["cp"]].pt / jets_coll["pythia_nonu"][pythia_nonu_to_cp["pythia_nonu"]].pt)
    ), bins=b, histtype="step", lw=1, label="CaloParticle vs. Pythia"
);

plt.hist(
    awkward.flatten(
        (jets_coll["cp"][pythia_to_cp["cp"]].pt / jets_coll["pythia"][pythia_to_cp["pythia"]].pt)
    ), bins=b, histtype="step", lw=1, label="CaloParticle vs Pythia NoNu"
);

plt.xscale("log")
plt.yscale("log")
plt.xlabel("jet $p_T$ / Pythia jet $p_T$")
plt.legend(loc=1, fontsize=12)
#plt.ylim(1,1e7)
plt.axvline(1.0, color="black", ls="--", lw=0.5)

In [ ]:
plt.figure(figsize=(5,5))
b = np.linspace(0.9,1.1,200)


pt_pythia = jets_coll["pythia_nonu"][pythia_nonu_to_cp["pythia_nonu"]].pt
pt_cp = jets_coll["cp"][pythia_nonu_to_cp["cp"]].pt
plt.hist(
    awkward.flatten(
        (pt_cp / pt_pythia)[pt_pythia>50]
    ), bins=b, histtype="step", lw=1, label="CaloParticle vs. Pythia"
);

pt_pythia = jets_coll["pythia"][pythia_to_cp["pythia"]].pt
pt_cp = jets_coll["cp"][pythia_to_cp["cp"]].pt
plt.hist(
    awkward.flatten(
        (pt_cp / pt_pythia)[pt_pythia>50]
    ), bins=b, histtype="step", lw=1, label="CaloParticle vs Pythia NoNu"
);

#plt.xscale("log")
plt.yscale("log")
plt.xlabel("jet $p_T$ / Pythia jet $p_T$")
plt.legend(loc=1, fontsize=12)
#plt.ylim(1,1e7)
plt.axvline(1.0, color="black", ls="--", lw=0.5)

## MET

In [ ]:
def met(pt, phi):
    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pt = np.sqrt(awkward.sum(px, axis=1)**2 + awkward.sum(py, axis=1)**2)
    return pt

met_pythia_nonu = met(particles_pythia[mask_pythia_nonu]["gen_pt"], particles_pythia[mask_pythia_nonu]["gen_phi"])
met_cp = met(particles_cp[mask_cp]["caloparticle_pt"], particles_cp[mask_cp]["caloparticle_phi"])

In [ ]:
b = np.linspace(0,100,21)
plt.figure(figsize=(5,5))
plt.hist(genmet, bins=b, histtype="step", label="genMetTrue")
plt.hist(met_pythia_nonu, bins=b, histtype="step", label="Pythia, no nu")
plt.hist(met_cp, bins=b, histtype="step", label="CaloParticle")
#plt.xscale("log")
plt.yscale("log")
plt.legend(loc=1, fontsize=12)
plt.xlabel("MET [GeV]")

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(genmet, met_pythia_nonu)
plt.xlabel("genMetTrue")
plt.ylabel("Pythia MET")

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(met_pythia_nonu, met_cp)
plt.xlabel("Pythia MET")
plt.ylabel("CaloParticle MET")